#Install libraries

In [ ]:
!pip install tensorflow
!pip install transformers
!pip install datasets
!pip install nltk
!pip install scikit-learn

#Import libraries

In [ ]:
import numpy as np
import tqdm as tq
import random

import nltk
from datasets import Dataset

import tensorflow as tf
from tensorflow import keras
import string 

import tensorflow as tf
import tensorflow_addons as tfa
from transformers import AutoTokenizer, TFAutoModel
from sklearn.metrics import classification_report


nltk.download("punkt")
tf.get_logger().setLevel(logging.ERROR)

#Initializations

In [ ]:
PRETRAINED_MODEL = ""

#Preprocessing

In [ ]:
"Write your preprocessing code here"

#Dataloader

In [ ]:
def data_loader():
  "Write your data loader code here"

  return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = data_loader()

#Embedding model

In [ ]:
def create_model(lr):
  
  inputs = tf.keras.layers.Input(shape=(512,), dtype=np.int64)
  y = TFAutoModel.from_pretrained(PRETRAINED_MODEL)(inputs)['pooler_output']
  y = tf.keras.layers.Dense(128, activation=None)(y)
  y = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(y)
  model = tf.keras.Model(inputs, y)

  model.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss=tfa.losses.TripletSemiHardLoss())
  
  return model

es_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model = create_model(2e-4)
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
          batch_size = 32, callbacks = [es_callback])

X_train_new = model.predict(X_train)
X_test_new = model.predict(X_test)

#Classifier model

In [ ]:
class FuzzyMMC:

    def __init__(self, sensitivity=1, exp_bound=1, animate=False):

        self.sensitivity = sensitivity
        self.hyperboxes = None
        self.isanimate = animate
        self.classes = np.array([])
        self.exp_bound = exp_bound

        if self.animate:
            self.box_history = []
            self.train_patterns = []


    def membership(self, pattern):

        min_pts = self.hyperboxes[:, 0, :]
        max_pts = self.hyperboxes[:, 1, :]

        a = np.maximum(0, (1 - np.maximum(0, (self.sensitivity * np.minimum(1, pattern - max_pts)))))
        b = np.maximum(0, (1 - np.maximum(0, (self.sensitivity * np.minimum(1, min_pts - pattern)))))

        return np.sum(a + b, axis=1) / (2 * len(pattern))


    def overlap_contract(self, index):

        contracted = False
        for test_box in range(len(self.hyperboxes)):

            if self.classes[test_box] == self.classes[index]:
                continue

            expanded_box = self.hyperboxes[index]
            box = self.hyperboxes[test_box]

            vj, wj = expanded_box
            vk, wk = box

            delta_new = delta_old = 1
            min_overlap_index = -1
            for i in range(len(vj)):
                if vj[i] < vk[i] < wj[i] < wk[i]:
                    delta_new = min(delta_old, wj[i] - vk[i])

                elif vk[i] < vj[i] < wk[i] < wj[i]:
                    delta_new = min(delta_old, wk[i] - vj[i])

                elif vj[i] < vk[i] < wk[i] < wj[i]:
                    delta_new = min(delta_old, min(wj[i] - vk[i], wk[i] - vj[i]))

                elif vk[i] < vj[i] < wj[i] < wk[i]:
                    delta_new = min(delta_old, min(wj[i] - vk[i], wk[i] - vj[i]))

                if delta_old - delta_new > 0:
                    min_overlap_index = i
                    delta_old = delta_new

            if min_overlap_index >= 0:
                i = min_overlap_index
                if vj[i] < vk[i] < wj[i] < wk[i]:
                    vk[i] = wj[i] = (vk[i] + wj[i])/2

                elif vk[i] < vj[i] < wk[i] < wj[i]:
                    vj[i] = wk[i] = (vj[i] + wk[i])/2

                elif vj[i] < vk[i] < wk[i] < wj[i]:
                    if (wj[i] - vk[i]) > (wk[i] - vj[i]):
                        vj[i] = wk[i]

                    else:
                        wj[i] = vk[i]

                elif vk[i] < vj[i] < wj[i] < wk[i]:
                    if (wk[i] - vj[i]) > (wj[i] - vk[i]):
                        vk[i] = wj[i]

                    else:
                        wk[i] = vj[i]

                self.hyperboxes[test_box] = np.array([vk, wk])
                self.hyperboxes[index] = np.array([vj, wj])
                contracted = True

        return contracted



    def train_pattern(self, X, Y):

        target = Y

        if target not in self.classes:

            if self.hyperboxes is not None:
                self.hyperboxes = np.vstack((self.hyperboxes, np.array([[X, X]])))
                self.classes = np.hstack((self.classes, np.array([target])))

            else:
                self.hyperboxes = np.array([[X, X]])
                self.classes = np.array([target])

            if self.isanimate:
                self.box_history.append(np.copy(self.hyperboxes))
                self.train_patterns.append((X, Y))
        else:

            memberships = self.membership(X)
            memberships[np.where(self.classes != target)] = 0
            memberships = sorted(list(enumerate(memberships)), key=lambda x: x[1], reverse=True)

            count = 0
            while True:
                index = memberships[count][0]
                min_new = np.minimum(self.hyperboxes[index, 0, :], X)
                max_new = np.maximum(self.hyperboxes[index, 1, :], X)

                if self.exp_bound * len(np.unique(self.classes)) >= np.sum(max_new - min_new):
                    self.hyperboxes[index, 0] = min_new
                    self.hyperboxes[index, 1] = max_new
                    break
                else:
                    count += 1

                if count == len(memberships):
                    self.hyperboxes = np.vstack((self.hyperboxes, np.array([[X, X]])))
                    self.classes = np.hstack((self.classes, np.array([target])))
                    index = len(self.hyperboxes) - 1
                    break

            if self.isanimate:
                self.box_history.append(np.copy(self.hyperboxes))
                self.train_patterns.append((X, Y))

            contracted = self.overlap_contract(index)

            if self.isanimate and contracted:
                self.box_history.append(np.copy(self.hyperboxes))
                self.train_patterns.append((X, Y))


    def fit(self, X, Y):

        for x, y in zip(X, Y):
            self.train_pattern(x, y)


    def predict(self, X):

        classes = np.unique(self.classes)
        results = []
        memberships = self.membership(X)
        max_prediction = 0
        pred_class = 0
        for _class in classes:
            mask = np.zeros((len(self.hyperboxes),))
            mask[np.where(self.classes == _class)] = 1
            p = memberships * mask
            prediction, class_index = np.max(p), np.argmax(p)
            if prediction > max_prediction:
                max_prediction = prediction
                pred_class = class_index

        return max_prediction, self.classes[pred_class]


    def score(self, X, Y):

        count = 0
        for x, y in zip(X, Y):
            _, pred = self.predict(x)
            if y == pred:
                count += 1

        return count / len(Y)


#Performance

In [ ]:
classifier = FuzzyMMC()
classifier.fit(X_train_new, y_train)
_, preds = classifier.predict(X_test_new)

print(classification_report(y_test, preds))